In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,SimpleRNN
import numpy as np
import pandas as pd
import copy
df = pd.read_csv("../Data/data_8_nodes_yen.csv") #You need to change #directory accordingly
df.head()

X(1, 2)  X(1, 3)  X(2, 4)  X(2, 5)  X(3, 4)  X(3, 7)  X(4, 6)  X(5, 6)  \
0       10       40       30       50       30       20       20       10   
1       90       50       50       30       30       80       80       30   
2       10       60       80       50       40       50       30       90   
3       80       20       40       60       50       80       90       60   
4       20       70       50       40       20       70       80       80   

   X(5, 8)  X(6, 7)  ...  Y4(1, 3)  Y4(2, 4)  Y4(2, 5)  Y4(3, 4)  Y4(3, 7)  \
0       90       70  ...         0         1         0         0         0   
1       30       80  ...         1         0         0         1         0   
2       80       30  ...         1         0         0         1         0   
3       80       80  ...         1         0         0         1         0   
4       10       10  ...         0         1         0         0         0   

   Y4(4, 6)  Y4(5, 6)  Y4(5, 8)  Y4(6, 7)  Y4(7, 8)  
0         1         0         0         1         1  
1         1         1         1         0         0  
2         1         0         0         1         1  
3         1         1         1         0         0  
4         1         1         1         0         0  

[5 rows x 55 columns]

In [2]:
edges = [[1, 2], [1, 3], [2, 4], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 7], [7, 8]]

In [3]:
number_edges = 11
number_steps = 1
start_node = 1
end_node = 8

In [4]:
X = df.iloc[:10000,:number_edges].values
y = df.iloc[:10000,number_edges:(number_edges*(number_steps+1))].values

In [5]:
X.shape

(10000, 11)

In [6]:
y.shape

(10000, 11)

In [7]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [8]:
X2[1]

array([40, 20, 90, 50, 30, 80, 10, 80, 20, 80, 30], dtype=int64)

In [9]:
y_test[1]

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [11]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(9000, 1, 11)

In [12]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(1000, 1, 11)

In [13]:
y_train[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [14]:
y_train.shape

(9000, 11)

In [15]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(9000, 1, 11)

In [16]:
y_train_2[0]

array([[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]], dtype=int64)

In [17]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(1000, 1, 11)

In [18]:
X2[1,:]

array([40, 20, 90, 50, 30, 80, 10, 80, 20, 80, 30], dtype=int64)

In [19]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [20]:
class Network:
    def __init__(self):
        self.model = Sequential()
        self.model.add(SimpleRNN(16, input_shape=(number_steps, number_edges), return_sequences=True))
        self.model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))
        self.model.compile(loss='BinaryCrossentropy', optimizer='sgd')
        self.fitness = float('inf')
        self.fitness_vector = 0
        self.counter = 0
        self.prob = 0
  
    def set_weights(self, weights):
        self.model.set_weights(weights) 
    
    def get_weights(self):
        return self.model.get_weights()
    
    def architecture(self):
        self.model.summary()
    
    def train(self):
        self.model.fit(X_train, y_train_2, epochs=100, batch_size=64)
    
    def evaluate(self):
        loss = self.model.evaluate(X_train, y_train_2)
        return loss
    
    def get_accuracy(self):
        pred = self.model.predict(X_train)
        accuracy = np.zeros(number_steps+1)
        for i in range(pred.shape[0]):
            condition_1 = True
            for t in range(number_steps):
                output = copy.deepcopy(pred[i][t])
                condition_2 = True
                for j in range(len(output)):
                    output[j] = round(output[j])
                if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
                    condition_2 = False
                    condition_1 = False
                if condition_2:
                    accuracy[t] += 1
            if condition_1:
                accuracy[number_steps] += 1
        accuracy = np.round(accuracy / pred.shape[0] * 100,2)
        return accuracy[-1]

In [21]:
class ABC:
    def __init__(self, N, Max):
        self.N = N
        self.Max = Max
        self.population = [Network() for i in range(self.N)]
        self.best = Network()
        self.children_population_weights = []
        self.shapes = [a.shape for a in Network().get_weights()]
        self.limit = self.N
    
    def Train_generation(self):
        i =  0
        for member in self.population:
            print("Train for member: ",i)
            i+=1
            member.train()
    
    def Train_best(self):
        self.best.train()
    
    def Flatten(self, W):
        flattened = np.concatenate([a.flatten() for a in W])
        return flattened
    
    def Un_Flatten(self, flattened):
        newarray = []
        index = 0
        for shape in self.shapes:
            size = np.product(shape)
            newarray.append(flattened[index : index + size].reshape(shape))
            index += size
        return newarray
    
    def Calculate_fitness(self):
        for i in range(len(self.population)):
            self.population[i].fitness = self.population[i].evaluate()
            
    def EmployeedPhase(self):
        self.children_population_weights.clear()
        counter = [a.counter for a in self.population]
        fitness = [a.fitness for a in self.population]
        for i in range(self.N):
            r = list(range(0,i)) + list(range(i+1,self.N))
            c = np.random.choice(r)
            coceg_w0 = self.population[c].get_weights()
            coceg = self.Flatten(coceg_w0)
            original_w0 = self.population[i].get_weights()
            original = self.Flatten(original_w0)
            d = np.random.randint(len(original))
            fi = np.random.uniform(-1,1)
            original[d] = original[d]+fi*(original[d]-coceg[d])
            origina_w = self.Un_Flatten(original)
            solution = Network()
            solution.set_weights(origina_w)
            solution.fitness = solution.evaluate()
            if(solution.fitness < fitness[i]):
                counter[i] = 0
                fitness[i] = solution.fitness
                self.children_population_weights.append(origina_w)
            else:
                counter[i]+=1
                self.children_population_weights.append(original_w0)
        for i in range(self.N):
            self.population[i].set_weights(self.children_population_weights[i])
            self.population[i].fitness = fitness[i]
            self.population[i].counter = counter[i]
    
    def OnlookedPhase(self):
        for i in range(self.N):
            self.population[i].fitness_vector = 1/self.population[i].fitness
        sum = 0
        for i in range(self.N):
            sum+=self.population[i].fitness_vector
        for i in range(self.N):
            self.population[i].prob=self.population[i].fitness_vector/sum
        prob=[a.prob for a in self.population]
        for i in range(self.N):
            index_solution = np.random.choice(list(range(self.N)),p=prob)
            r = list(range(0,index_solution)) + list(range(index_solution+1,self.N))
            c = np.random.choice(r)
            coceg_w0 = self.children_population_weights[c]
            coceg = self.Flatten(coceg_w0)
            original_w0 = self.children_population_weights[index_solution]
            original = self.Flatten(original_w0)
            d = np.random.randint(len(original))
            fi = np.random.uniform(-1,1)
            original[d] = original[d]+fi*(original[d]-coceg[d])
            origina_w = self.Un_Flatten(original)
            solution = Network()
            solution.set_weights(origina_w)
            solution.fitness = solution.evaluate()
            if(solution.fitness < self.population[index_solution].fitness):
                self.population[index_solution].counter = 0
                self.population[index_solution].fitness = solution.fitness
                self.population[index_solution].set_weights(origina_w)
            else:
                self.population[index_solution].counter +=1
        
    def ScoutPhase(self):
        for i in range(self.N):
            if self.population[i].counter > self.limit:
                self.population[i] = Network()
    
    def GetBest(self):
        self.population.sort(key=lambda x: x.fitness)
        if self.population[0].fitness < self.best.fitness:
            self.best.set_weights(self.population[0].get_weights())
            self.best.fitness = self.population[0].fitness

    def Do(self):
        self.Calculate_fitness()
        for i in range(self.Max):
            print("i = ",i)
            self.EmployeedPhase()
            self.OnlookedPhase()
            self.ScoutPhase()
            self.GetBest()
            print("best fitness:",self.best.fitness)

In [22]:
N = 10
Max = 100

In [23]:
abc = ABC(N, Max)
abc.Do()

282/282 [==============================] - 1s 1ms/step - loss: 0.7327
i =  0
282/282 [==============================] - 1s 1ms/step - loss: 0.7540
best fitness: 0.7078892588615417
i =  1
282/282 [==============================] - 1s 1ms/step - loss: 0.7327
best fitness: 0.7078892588615417
i =  2
282/282 [==============================] - 1s 1ms/step - loss: 0.7424
best fitness: 0.7078892588615417
i =  3
282/282 [==============================] - 1s 1ms/step - loss: 0.7078
best fitness: 0.707791805267334
i =  4
282/282 [==============================] - 1s 1ms/step - loss: 0.7077
best fitness: 0.707737386226654
i =  5
282/282 [==============================] - 1s 1ms/step - loss: 0.7364


282/282 [==============================] - 1s 1ms/step - loss: 0.7353
best fitness: 0.7076590657234192
i =  6
282/282 [==============================] - 1s 1ms/step - loss: 0.7423
best fitness: 0.7076485753059387
i =  7
282/282 [==============================] - 1s 1ms/step - loss: 0.7326
best fitness: 0.7076485753059387
i =  8
282/282 [==============================] - 1s 1ms/step - loss: 0.7725
best fitness: 0.7076485753059387
i =  9
282/282 [==============================] - 1s 1ms/step - loss: 0.7340
best fitness: 0.7076485753059387
i =  10
282/282 [==============================] - 1s 1ms/step - loss: 0.7357


282/282 [==============================] - 1s 1ms/step - loss: 0.7076
best fitness: 0.7072873115539551
i =  11
282/282 [==============================] - 1s 1ms/step - loss: 0.7356
best fitness: 0.7046395540237427
i =  12
282/282 [==============================] - 1s 1ms/step - loss: 0.7725
best fitness: 0.7046395540237427
i =  13
282/282 [==============================] - 1s 1ms/step - loss: 0.7320
best fitness: 0.7046395540237427
i =  14
282/282 [==============================] - 1s 1ms/step - loss: 0.7422
best fitness: 0.7046395540237427
i =  15
282/282 [==============================] - 1s 1ms/step - loss: 0.7421
best fitness: 0.7046395540237427
i =  16
282/282 [==============================] - 1s 2ms/step - loss: 0.7317


282/282 [==============================] - 1s 1ms/step - loss: 0.7195
best fitness: 0.7034755945205688
i =  17
282/282 [==============================] - 1s 1ms/step - loss: 0.7405
best fitness: 0.7034755945205688
i =  18
282/282 [==============================] - 1s 1ms/step - loss: 0.7035
best fitness: 0.7034645080566406
i =  19
282/282 [==============================] - 1s 1ms/step - loss: 0.7395
best fitness: 0.7031856775283813
i =  20
282/282 [==============================] - 1s 1ms/step - loss: 0.7654
best fitness: 0.7031856775283813
i =  21
282/282 [==============================] - 1s 2ms/step - loss: 0.7388
best fitness: 0.7025202512741089
i =  22
282/282 [==============================] - 1s 2ms/step - loss: 0.7384


282/282 [==============================] - 1s 1ms/step - loss: 0.7383
best fitness: 0.7008857131004333
i =  23
282/282 [==============================] - 1s 1ms/step - loss: 0.7649
best fitness: 0.7008857131004333
i =  24
282/282 [==============================] - 1s 2ms/step - loss: 0.7009
best fitness: 0.6967183947563171
i =  25
282/282 [==============================] - 1s 1ms/step - loss: 0.7524
best fitness: 0.6942703127861023
i =  26
282/282 [==============================] - 1s 1ms/step - loss: 0.7726
best fitness: 0.6942703127861023
i =  27
282/282 [==============================] - 1s 2ms/step - loss: 0.7714


...

282/282 [==============================] - 1s 1ms/step - loss: 0.7436
best fitness: 0.6739697456359863
i =  74
282/282 [==============================] - 1s 2ms/step - loss: 0.6740
best fitness: 0.6739627122879028
i =  75
282/282 [==============================] - 1s 1ms/step - loss: 0.7239
best fitness: 0.6739627122879028
i =  76
282/282 [==============================] - 1s 1ms/step - loss: 0.7119
best fitness: 0.6739627122879028
i =  77
282/282 [==============================] - 1s 1ms/step - loss: 0.7466
best fitness: 0.6739612221717834
i =  78
282/282 [==============================] - 1s 1ms/step - loss: 0.7216


282/282 [==============================] - 1s 1ms/step - loss: 0.7401
best fitness: 0.6739612221717834
i =  79
282/282 [==============================] - 1s 1ms/step - loss: 0.7752
best fitness: 0.6739612221717834
i =  80
282/282 [==============================] - 1s 1ms/step - loss: 0.7228
best fitness: 0.6739612221717834
i =  81
282/282 [==============================] - 1s 1ms/step - loss: 0.7228
best fitness: 0.6739612221717834
i =  82
282/282 [==============================] - 1s 2ms/step - loss: 0.7745
best fitness: 0.6739612221717834
i =  83
282/282 [==============================] - 1s 2ms/step - loss: 0.7494
best fitness: 0.6739612221717834
i =  84
282/282 [==============================] - 1s 1ms/step - loss: 0.6742


282/282 [==============================] - 1s 1ms/step - loss: 0.7452
best fitness: 0.6739227175712585
i =  85
282/282 [==============================] - 1s 1ms/step - loss: 0.7475
best fitness: 0.6735180020332336
i =  86
282/282 [==============================] - 1s 1ms/step - loss: 0.6730
best fitness: 0.6729835867881775
i =  87
282/282 [==============================] - 1s 1ms/step - loss: 0.6730
best fitness: 0.6729835867881775
i =  88
282/282 [==============================] - 1s 1ms/step - loss: 0.7210
best fitness: 0.6729835867881775
i =  89
282/282 [==============================] - 1s 1ms/step - loss: 0.6735
best fitness: 0.6729835867881775
i =  90
282/282 [==============================] - 1s 1ms/step - loss: 0.7346


282/282 [==============================] - 1s 1ms/step - loss: 0.7062
best fitness: 0.6729835867881775
i =  91
282/282 [==============================] - 1s 2ms/step - loss: 0.7466
best fitness: 0.6729835867881775
i =  92
282/282 [==============================] - 0s 1ms/step - loss: 0.7465
best fitness: 0.6729835867881775
i =  93
282/282 [==============================] - 2s 3ms/step - loss: 0.7708
best fitness: 0.6729835867881775
i =  94
282/282 [==============================] - 1s 1ms/step - loss: 0.7337
best fitness: 0.6729835867881775
i =  95
282/282 [==============================] - 1s 2ms/step - loss: 0.7043


282/282 [==============================] - 1s 1ms/step - loss: 0.7404
best fitness: 0.6729835867881775
i =  96
282/282 [==============================] - 1s 1ms/step - loss: 0.7376
best fitness: 0.6729835867881775
i =  97
282/282 [==============================] - 1s 1ms/step - loss: 0.7699
best fitness: 0.6729835867881775
i =  98
282/282 [==============================] - 1s 1ms/step - loss: 0.7677
best fitness: 0.6729835867881775
i =  99
282/282 [==============================] - 0s 1ms/step - loss: 0.7437
best fitness: 0.6729835867881775


In [24]:
abc.best.train()

Epoch 1/100
141/141 [==============================] - 1s 1ms/step - loss: 0.6555
Epoch 2/100
141/141 [==============================] - 0s 1ms/step - loss: 0.6222
Epoch 3/100
141/141 [==============================] - 0s 1ms/step - loss: 0.5915
Epoch 4/100
141/141 [==============================] - 0s 1ms/step - loss: 0.5630
Epoch 5/100
141/141 [==============================] - 0s 1ms/step - loss: 0.5365
Epoch 6/100
141/141 [==============================] - 0s 1ms/step - loss: 0.5118
Epoch 7/100
141/141 [==============================] - 0s 1ms/step - loss: 0.4886
Epoch 8/100
141/141 [==============================] - 0s 1ms/step - loss: 0.4670
Epoch 9/100
141/141 [==============================] - 0s 1ms/step - loss: 0.4467
Epoch 10/100
141/141 [==============================] - 0s 1ms/step - loss: 0.4276
Epoch 11/100
141/141 [==============================] - 0s 1ms/step - loss: 0.4097
Epoch 12/100
141/141 [==============================] - 0s 1ms/step - loss: 0.3928
Epoch 13/100


141/141 [==============================] - 0s 1ms/step - loss: 0.1209
Epoch 100/100
141/141 [==============================] - 0s 1ms/step - loss: 0.1205


In [25]:
abc.best.get_accuracy()

282/282 [==============================] - 0s 945us/step


93.16

In [26]:
abc.best.evaluate()

282/282 [==============================] - 0s 1ms/step - loss: 0.1203


0.12028728425502777